# WI-Projekt - Untersuchung der Leistungsfähigkeit von KI-gestützten Datenbankabfragen

## Setup

#### Get required Packages

In [1]:
import json
import os

from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

credentials = "credentials.json"

with open(credentials, 'r') as f:
    credentials = json.load(f)

# API-Schlüssel setzen
os.environ["OPENAI_API_KEY"] = credentials['openai']['api_key'] # OpenAI

#### Connect to DB

In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///DB\Chinook.db")
#db = SQLDatabase.from_uri("sqlite:///DB\CarDealer.db")

print(db.dialect)
print(db.get_usable_table_names())

db.run("SELECT * FROM Artist LIMIT 10;") # Chinook
#db.run("SELECT * FROM cars LIMIT 10;") #Car dealer

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'AntÃ´nio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

### Setup Dynamic few-shot-prompt (vektorDB)

#### Load Examples

In [3]:
# Laden der Beispiele aus der seperaten Datei
with open('Dynamic few-shot-prompts/few-shot-prompts.json', 'r') as f:
    examples = json.load(f)


#### Search based on semantic similarity

In [4]:
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

# Auswählen passender Prompt-Beispiele
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    FAISS,
    k=5,
    input_keys=["input"],
)

#### Create FewShotPromptTemplate

In [5]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
    SystemMessagePromptTemplate,
)

# Laden des template_prefix aus einer Textdatei
with open('Dynamic few-shot-prompts/template_prefix.txt', 'r') as f:
    system_prefix = f.read()

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=PromptTemplate.from_template(
        "User input: {input}\nSQL query: {query}"
    ),
    input_variables=["input", "dialect", "top_k"],
    prefix=system_prefix, 
    suffix="",
)


#### Create full prompt

In [6]:
full_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=few_shot_prompt),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [7]:
# Preview
prompt_val = full_prompt.invoke(
    {
        "input": "Gib mir den Namen des Albums, mit den höchsten Gesamteinnahmen aus Track-Verkäufen, sowie die summierten Einnahmen dieser Tracks und die Anzahl verkaufter Tracks des Albums, basierend auf dem durchschnittlichen Einzelpreis der Tracks.",
        "top_k": 5,
        "dialect": "SQLite",
        "agent_scratchpad": [],
    }
)
print(prompt_val.to_string())

System: You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't 

### Agent OpenAI

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0) ## Which Large Language model are we using?

OpenAI_agent = create_sql_agent(
    llm, 
    toolkit= SQLDatabaseToolkit(llm=llm, db=db), 
    prompt=full_prompt,
    agent_type='tool-calling', 
    top_k=50,
    temperature=0,
    verbose=True) 


### Execute

In [9]:
OpenAI_agent.invoke(
    {"input": "Zeige mir die Künstler und den Gesamtumsatz, aber nur für Künstler mit mehr als 100 verkauften Tracks, in absteigender Reihenfolge."}
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Artist, Invoice, InvoiceLine, Track'}`



CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Invoice" (
	"InvoiceId" INTEGER NOT NULL, 
	"CustomerId" INTEGER NOT NULL, 
	"InvoiceDate" DATETIME NOT NULL, 
	"BillingAddress" NVARCHAR(70), 
	"BillingCity" NVARCHAR(40), 
	"BillingState" NVARCHAR(40), 
	"BillingCountry" NVARCHAR(40), 
	"BillingPostalCode" NVARCHAR(10), 
	"Total" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("InvoiceId"), 
	FOREIGN KEY("CustomerId") REFERENCES "Customer" ("CustomerId")
)

/*
3 rows from Invoice table:
InvoiceId	CustomerId	InvoiceDate	BillingAddress	BillingCity	BillingState	Billing

{'input': 'Zeige mir die Künstler und den Gesamtumsatz, aber nur für Künstler mit mehr als 100 verkauften Tracks, in absteigender Reihenfolge.',
 'output': 'Die Künstler mit einem Gesamtumsatz und mehr als 100 verkauften Tracks sind:\n\n1. Iron Maiden mit einem Gesamtumsatz von 1233,54\n2. U2 mit einem Gesamtumsatz von 895,59'}